<a href="https://colab.research.google.com/github/naenumtou/ifrs9/blob/main/PD/APIBOTMacro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set auto reload
%reload_ext autoreload
%autoreload 2

In [2]:
# Import libraries
import pandas as pd
import numpy as np

from http.client import HTTPSConnection
from dateutil.relativedelta import relativedelta

%config InlineBackend.figure_format = 'retina'

In [3]:
# Series code for macroecnomics variables
# The code can be found from BOT API
codes = [
        'EIMACROY00229',
        'EITRADERETM00384',
        'RLLFSWAGEM01158',
        'RLLFSWKM00079',
        'EILEIM00007',
        'EIPCIM00016',
        'EILEIM00013',
        'EILEIM00015',
        'EIPCIM00039',
        'EIPCIM00040',
        'EIPIIM00046',
        'EICEIM00003',
        'EICEIM00001',
        'XTEXIMPGX0M00599',
        'XTEXIMPGM0M00435',
        'MFFI00LTHHQ00904',
        'XTFDISCO00Q02980',
]

In [4]:
# Define BOT Parameters
APIKey = 'YOUR-KEY' #Hide for personal use
headers = {
    'X-IBM-Client-Id': APIKey,
    'accept': 'application/json'
}

In [5]:
# Define starting period
startPeriod = '2014-01-01' #Need ofrmat yyyy-mm-dd

# Define function to find prior year
def yearAgo(start, year):
  '''
  This is for macroeconomics variables transformation.
  It might require to use prior years data applied with formula.
  E.g. 12-Moving average for current month need 12 months before.
  '''
  start = pd.to_datetime(start)
  priorYear = start - relativedelta(years = year)
  return priorYear.strftime('%Y-%m-%d')

# The 2-years ago from startPeriod
print(f'2 years ago from start period: {yearAgo(startPeriod, 2)}')

2 years ago from start period: 2012-01-01


### BOT JSON Structure
![](https://i.postimg.cc/pXB9dgjd/Screen-Shot-2565-01-21-at-16-39-13.png)

In [6]:
# Connect to BOT over API
dataRequest = []

for code in codes:
  link = f'/bot/public/observations/?series_code={code}&start_period={startPeriod}'
  connect = HTTPSConnection('apigw1.bot.or.th') #Reconnect every loop
  connect.request(
      'GET',
      link,
      headers = headers
  )
  response = connect.getresponse() #HTTPResponse
  data = response.read()
  data = pd.io.json.loads(data.decode('utf-8')) #To JSON
  value = pd.DataFrame.from_dict(
      data['result']['series'][0]['observations'],
      orient = 'columns'
  )
  # Data basis manipulation
  value['period_start'] = pd.to_datetime(
      value['period_start'],
      dayfirst = True
  ) #To datetime
  if data['result']['series'][0]['frequency'] == 'Monthly':
    value['period_start'] = value['period_start'] + pd.offsets.MonthEnd()
  elif data['result']['series'][0]['frequency'] == 'Quarterly':
    value['period_start'] = value['period_start'] + pd.offsets.QuarterEnd()
  elif data['result']['series'][0]['frequency'] == 'Semi-annual':
    value['period_start'] = value['period_start'].str.replace('H', 'Q', regex = True)
    value['period_start'] = value['period_start'] + pd.offsets.QuarterEnd()
  else:
    value['period_start'] = value['period_start'] + pd.offsets.YearEnd()
  # Get name of code series
  value = value.rename(
      columns = {'value': data['result']['series'][0]['series_name_eng']}
  )
  # Set period to index for mapping
  value = value.set_index('period_start')
  dataRequest.append(value)

In [7]:
# To DataFrame
df = pd.concat(
    dataRequest,
    axis = 1,
    sort = True
)

# Show table result
df.head(5)

,Core Consumer Price Index (2015=100),Retail Sales Index,Total Average wage Monthly,Unemployment Rate Nationwide Monthly,Leading Economic Index,Private Consumption Index,SET Index,Oil Price Inverse Index (Dubai),Sales of Commercial cars (Seasonally Adjusted),Sales of Passenger cars (Seasonally Adjusted),Private Investment Index (PII),Manufacturing Production Index,Coincident Economic Index,"Exports, f.o.b. (BOP basis) Millions of Baht","Imports, f.o.b. (BOP basis) Millions of Baht",Loans to Household Total,Foreign Direct Investment Position United States Millions of USD
period_start,,,,,,,,,,,,,,,,,
2014-01-31,NaN,212.7600000,12911.2400000,0.94,134.8900000,111.3700000,1274.2800000,0.9600000,40842.3200000,35761.7400000,119.3300000,96.3000000,124.0700000,587085.5926191,637677.4936245,NaN,NaN
2014-02-28,NaN,206.7400000,12809.4400000,0.86,135.5500000,102.5100000,1325.3300000,0.9500000,38075.8100000,35836.4900000,115.9300000,95.4900000,122.6700000,598571.1516825,488289.3776387,NaN,NaN
2014-03-31,NaN,231.5500000,12633.1400000,0.88,135.4900000,118.6800000,1376.2600000,0.9600000,37545.8000000,35170.0600000,129.0600000,96.1200000,123.0300000,641085.6650691,553252.0792439,9988847.0000000,16687.2100000
2014-04-30,NaN,205.7900000,13219.2900000,0.90,135.4300000,116.4800000,1414.9400000,0.9600000,39942.1700000,39888.3800000,115.8200000,98.1100000,123.7200000,556218.7899483,559413.4982299,NaN,NaN
2014-05-31,NaN,214.2500000,13264.3900000,0.94,135.9900000,111.9300000,1415.7300000,0.9500000,38235.8700000,33759.2100000,122.6700000,97.2600000,123.2700000,631494.0812195,600311.6254149,NaN,NaN
